In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2



In [2]:
batch_size = 32
img_size = 128
WIDTH = 236
HEIGHT = 137
model = load_model('/kaggle/input/128-mixup/model.h5')
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax
def resize(df, img_size = 128, pad=16):
    resized = {}
    for i in range(df.shape[0]):
        image = 255 - df.loc[df.index[i]].values.reshape(137,236)
        ymin,ymax,xmin,xmax = bbox(image[5:-5,5:-5] > 60)
        xmin = xmin - 13 if (xmin > 13) else 0
        ymin = ymin - 10 if (ymin > 10) else 0
        xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
        ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
        image = image[ymin:ymax,xmin:xmax]
        image[image < 28] = 0
        lx, ly = xmax-xmin,ymax-ymin
        l = max(lx,ly) + pad
        image = np.pad(image, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
        image = cv2.resize(image,(img_size,img_size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

OSError: SavedModel file does not exist at: /kaggle/input/128-mixup/128_mixup_next_next.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}
components = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
target=[]
row_id=[]

In [ ]:
for i in range(4):
    print ('step')
    df_test_img = np.array_split((pd.read_parquet(f'/kaggle/input/bengaliai-cv19/test_image_data_{i}.parquet').set_index('image_id', drop=True).astype('uint8')) , 3)

    for s in range(3):
        print ('resize')
        X_test = np.divide(resize(df_test_img[s]), 255.0, dtype = 'float16')
        X_test = X_test.values.reshape(-1, img_size, img_size, 1)
        print ('predicting')
        preds = model.predict(x = X_test, batch_size = batch_size)

        del X_test

        for j, p in enumerate(preds_dict):
            preds_dict[p] = np.argmax(preds[j], axis=1)

        for k,id in enumerate(df_test_img[s].index.values):
            for i,comp in enumerate(components):
                id_sample=id+'_'+comp
                row_id.append(id_sample)
                target.append(preds_dict[comp][k])

In [ ]:
submit_df = pd.DataFrame({'row_id':row_id,'target':target}, columns = ['row_id','target'])
submit_df.to_csv('submission.csv', index = False)

In [ ]:
submit_df.head(50)